This is the code for the pipeline to get all the files, compare models on a sample, translate all files using the best model and correct proper nouns.

Workflow:
1. Extract information from the CONLL-U
2. Translate
3. Tokenize English translations with Stanza
4. Word alignment, substitute English NE translations with lemmas from the source, get information on NE annotations for each translated word from the source annotations
5. Linguistically process English translation with Stanza (lemmas, POS)
6. Parse CONLL-u file and add additional information (sentence ids, alignments, NER annotations)

# Functions

conllu_to_df

In [57]:
def conllu_to_df(parl_list, file_name_list, main_path, lang_code):
	"""
	Take the conllu files and extract relevant information. Save everything in a DataFrame.

	Args:
	- parl_list: list of documents with their entire paths to be included (see step above).
	- file_name_list: list of names of the files (see step above)
	- main_path: main path to the working directory
	- lang_code: e.g., CZ
	"""
	from conllu import parse
	import pandas as pd
	from IPython.display import display
	from itertools import islice
	import math
	import numpy as np

	# Create an empty df
	df = pd.DataFrame({"file_path": [""],"file": [""], "sentence_id": [""], "text": [""], "tokenized_text": [""], "proper_nouns": [""]})

	# Check whether there are any problems with parsing the documents
	"""
	
	error_count = 0
	problematic_doc_list = []

	for doc in parl_list:
		try:
			# Open the file
			data = open("{}".format(doc), "r").read()

			sentences = parse(data)
		except:
			error_count += 1
			problematic_doc_list.append(doc)

	print(error_count)
	print(problematic_doc_list)
	"""
	# Parse the data with CONLL-u parser
	for doc in parl_list:
		# Open the file
		data = open("{}".format(doc), "r").read()
		
		sentences = parse(data)

		sentence_id_list = []
		text_list = []
		tokenized_text_list = []
		proper_noun_list = []

		for sentence in sentences:
			# Find sentence ids
			current_sentence_id = sentence.metadata["sent_id"]
			sentence_id_list.append(current_sentence_id)

			# Find text - if texts consists of multiword tokens, these tokens will appear as they are,
			# not separated into subwords
			current_text = sentence.metadata["text"]
			text_list.append(current_text)

			# Create a string out of tokens
			current_token_list = []
			word_dict = {}

			for token in sentence:
				# Find multiword tokens and take their NER
				if type(token["id"]) != int:
					multiword_ner = token["misc"]["NER"]
				
				else:
				# Append to the tokenized text tokens that are not multiword tokens
				# (we append subtokens to the tokenized texts, not multiword tokens)
					current_token_list.append(token["form"])
					

					# Create a list of NE annotations with word indices.
					# I'll substract one from the word index,
					# because indexing in the CONLLU file starts with 1, not 0
					current_index = int(token["id"]) - 1

					# If the word does not have NER annotation,
					# take the annotation from the multiword token
					if token["misc"] is None:
						current_ner = multiword_ner
					else:
						current_ner = token["misc"]["NER"]

					# Add information on the lemma if the NE is personal name
					if "PER" in current_ner:
						word_dict[current_index] = [token["form"], token["lemma"]]

			proper_noun_list.append(word_dict)

			current_string = " ".join(current_token_list)

			tokenized_text_list.append(current_string)

		
		new_df = pd.DataFrame({"sentence_id": sentence_id_list, "text": text_list, "tokenized_text": tokenized_text_list, "proper_nouns": proper_noun_list})

		new_df["file_path"] = doc

		# Get the file name
		file_name = file_name_list[parl_list.index(doc)]
		new_df["file"] = file_name

		# Merge df to the previous df
		df = pd.concat([df, new_df])
	
	# Reset index
	df = df.reset_index(drop=True)

	# Remove the first row
	df = df.drop([0], axis="index")

	# Reset index
	df = df.reset_index(drop=True)

	# Add information on length
	df["length"] = df["text"].str.split().str.len()

	print("Number of words in the corpora: {}".format(df["length"].sum()))

	# Split the dataframe into 3 batches based on the list of files
	file_list = list(df.file.unique())

	def chunk(arr_range, arr_size):
		arr_range = iter(arr_range)
		return iter(lambda: tuple(islice(arr_range, arr_size)), ())

	batches_list = list(chunk(file_list, math.ceil(len(file_list)/3)))

	print("File is separated into {} batches, sizes of batches (in no. of files): {}, {}, {}.".format(len(batches_list), len(batches_list[0]), len(batches_list[1]), len(batches_list[2])))

	# Add information on the batch in the dataframe
	for i in range(len(batches_list)):
		if i==0:
			df["batch"] = np.where((df["file"].isin(list(batches_list[i]))), int(i+1), "none")
		else:
			df["batch"] = np.where((df["file"].isin(list(batches_list[i]))), int(i+1), df["batch"])

	extracted_dataframe_path = "{}/results/{}/ParlaMint-{}-extracted-source-data.csv".format(main_path, lang_code, lang_code)

	# Save the dataframe
	df.to_csv("{}".format(extracted_dataframe_path), sep="\t")

	print("Dataframe saved as {}".format(extracted_dataframe_path))
	
	# Show the results
	print(df.describe(include="all").to_markdown())

	# Save each batch separately to be translated separately
	for i in list(df.batch.unique()):
		df[df["batch"] == i].to_csv("{}.{}.csv".format(extracted_dataframe_path, i), sep="\t")
		print("Batch {} saved as {}.{}.csv".format(i, extracted_dataframe_path, i))

	return df

choose_model

In [2]:
def choose_model(lang_code, main_path):
	"""
	Compare a small sample of translations of all OPUS-MT models that are available
	for the language, to decide which one to use. The function prints out a dataframe with all translations of the sample and saves it as ParlaMint-{lang_code}-sample-model-comparison.csv.

	Args:
	- lang_code: the lang code that is used in the names of the files, it should be the same as for extract_text()
	"""
	import pandas as pd
	import regex as re
	from easynmt import EasyNMT
	from IPython.display import display
	
	lang_models_dict = {"BG": ["bg", "sla", "zls"], "HR": ["zls"], "CZ": ["cs", "sla", "zlw" ], "DK": ["da", "gmq", "gem"], "NL": ["nl", "gem", "gmw"], "FR": ["fr", "itc","roa"], "HU": ["mul"], "IS": ["is","gmq", "gem"], "IT": ["it", "roa", "itc"], "LV": ["lv","bat"], "LT": ["bat"], "PL": ["pl", "sla", "zlw"], "SI": ["sla", "zls"], "ES": ["es", "roa", "itc"], "TR": ["tr", "trk" ], "AT": ["de", "gem", "gmw"], "ES-PV": ["eu", "mul"], "BA": ["sla", "zls"], "ES-CT": ["ca", "roa", "itc"], "EE": ["et", "urj", "fiu"], "FI": ["fi", "urj", "fiu"], "ES-GA": ["gl", "roa", "itc"], "GR": ["grk"], "PT": ["roa", "itc"], "RO":["roa", "itc"], "RS": ["zls", "sla"], "SE": ["sv", "gmq", "gem"], "UA":["uk", "sla", "zle"]}

	extracted_dataframe_path = "{}/results/{}/ParlaMint-{}-extracted-source-data.csv".format(main_path, lang_code, lang_code)

	# Open the file, created in the previous step
	df = pd.read_csv("{}".format(extracted_dataframe_path), sep="\t", index_col=0)

	# Define the model
	model = EasyNMT('opus-mt')

	print("Entire corpus has {} sentences and {} words.".format(df["text"].count(), df["length"].sum()))

	# Create a smaller sample - just a couple of sentences from one file
	df = df[df.file == list(df["file"].unique())[0]][:20]

	print("Sample files has {} sentences and {} words.".format(df["text"].count(), df["length"].sum()))

	# Create a list of sentences from the df
	sentence_list = df.text.to_list()

	# Translate the sample using all available models for this language
	for opus_lang_code in lang_models_dict[lang_code]:
		translation_list = model.translate(sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

		# Add the translations to the df
		df["translation-{}".format(opus_lang_code)] = translation_list
	
	df = df.drop(columns=["file", "sentence_id", "tokenized_text", "proper_nouns", "length"])

	# Save the df
	df.to_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-sample-model-comparison.csv".format(lang_code, lang_code))

	print("The file is saved as/home/tajak/Parlamint-translation/ results/{}/ParlaMint-{}-sample-model-comparison.csv. ".format(lang_code, lang_code))

	return df


translate()

In [5]:
def translate(lang_code, opus_lang_code, main_path):
	"""
	This function translates the text from the dataframe, created with the extract_text() function
	with OPUS-MT models using EasyNMT. It returns a dataframe with the translation.

	Args:
	- lang_code: the lang code that is used in the names of the files, it should be the same as for extract_text()
	- opus_lang_code: the lang code to be used in the OPUS-MT model - use the one that performed the best in the comparison (see function choose_model())
	"""
	import pandas as pd
	import regex as re
	from easynmt import EasyNMT
	from IPython.display import display
	import time

	extracted_dataframe_path = "{}/results/{}/ParlaMint-{}-extracted-source-data.csv".format(main_path, lang_code, lang_code)

	translated_dataframe_path = "{}/results/{}/ParlaMint-{}-translated.csv".format(main_path, lang_code, lang_code)

	# Open the file, created in the previous step
	df = pd.read_csv("{}".format(extracted_dataframe_path), sep="\t", index_col=0)

	# Define the model
	model = EasyNMT('opus-mt')

	print("Entire corpus has {} sentences and {} words.".format(df["text"].count(), df["length"].sum()))

	# Create a list of sentences from the df
	sentence_list = df.text.to_list()

	print("Translation started.")

	start_time = time.time()

	#Translate the list of sentences - you need to provide the source language as it is in the name of the model - the opus_lang_code
	translation_list = model.translate(sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

	translation_time = round((time.time() - start_time)/60,2)

	print("Translation completed. It took {} minutes for {} instances - {} minutes per one sentence.".format(translation_time, len(sentence_list), translation_time/len(sentence_list)))

	# Add the translations to the df
	df["translation"] = translation_list

	# Display the df
	display(df[:3])

	# Save the df
	df.to_csv("{}".format(translated_dataframe_path), sep="\t")

	print("The file is saved as {}".format(translated_dataframe_path))

	return df

tokenize_translation()

In [6]:
def tokenize_translation(lang_code, main_path):
	import stanza

	nlp = stanza.Pipeline(lang='en', processors='tokenize', tokenize_no_ssplit = True)

	# Apply tokenization to English translation and add the sentences to the df
	# Open the df
	translated_dataframe_path = "{}/results/{}/ParlaMint-{}-translated.csv".format(main_path, lang_code, lang_code)

	df = pd.read_csv("{}".format(translated_dataframe_path), sep="\t", index_col = 0)

	# Save also the information on whether there is a space after or before punctuation
	# which we will need later, to remove unnecessary spaces
	En_sentences = df.translation.to_list()

	tokenized_sentences = []
	space_after_list = []

	for i in En_sentences:
		doc = nlp(i).to_dict()
		current_sentence_list = []
		current_space_after_list = []

		# Define a list of start_char and end_char
		start_chars = []
		end_chars = []

		# Loop through the tokens in the sentence and add them to a current sentence list
		for sentence in doc:
			for word in sentence:
				current_sentence_list.append(word["text"])

				# Add information on start and end chars to the list
				start_chars.append(word["start_char"])
				end_chars.append(word["end_char"])
			
		# Now loop through the start_char and end_char lists and find instances
		# where the end_char of one word is the same as the start_char of the next one
		# this means there is no space between them
		for char_index in range(len(start_chars)-1):
			if end_chars[char_index] == start_chars[(char_index+1)]:
				current_space_after_list.append("No")
			else:
				current_space_after_list.append("Yes")

		# This loop is not possible for the end token, so let's add information for the last token
		# just to avoid errors due to different lengths of lists
		current_space_after_list.append("Last")

		# Join the list into a space-separated string
		current_string = " ".join(current_sentence_list)

		tokenized_sentences.append(current_string)

		space_after_list.append(current_space_after_list)

	# Add the result to the df
	df["translation-tokenized"] = tokenized_sentences
	df["space-after-information"] = space_after_list

	translated_tokenized_dataframe_path = "{}/results/{}/ParlaMint-{}-translated-tokenized.csv".format(main_path,lang_code, lang_code)
	# Save the df
	df.to_csv("{}".format(translated_tokenized_dataframe_path), sep="\t")
	
	print("File saved as {}".format(translated_tokenized_dataframe_path))
	
	return df

alignment_file_to_target_dict()

In [7]:
# Create a dictionary from the returned alignment files which will be added to each word in the final conllu

def alignment_file_to_target_dict(file):
	"""
	The output of the eflomal aligner is in the source to target direction. We want to get the alignments in the other direction
	and for each target word add to the conllu its aligned source word index (as it appears in conllu). In conllu, indices start
	with 1, not 0. So, we take the eflomal files, reverse the order and create dictionaries with target indexes as keys
	and source indexes as values. If there are more than one words aligned to the same target word, it looks like this: '1, 2'.
	We use the conllu indexes which means that we add 1 to each index in the alingment pairs. 

	Args:
		- file: the path to the .fwd and .rev file that is produced by the eflomal tool

	The result is a list of dictionaries, each dictionary corresponds to one sentence.
	"""
	# Create target alignments from the source alignment direction (by changing the direction in the file)
	aligns_list_target = open(file, "r").readlines()
	aligns_list_target = [i.replace("\n", "") for i in aligns_list_target]
	aligns_list_target = [i.split(" ") for i in aligns_list_target]

	aligns_list_target_dict_list = []

	# Loop through the alignments for sentences
	for i in aligns_list_target:
		# Create a dictionary for each sentence
		current_sentence_align = {}
		# For each alignment pair in the sentence:
		for pair in i:
			# Split the pair: result is a list of lists with source index as the first element
			# and target index as the second element: [[0,0], [1,2], [1,3]]
			current_pair = pair.split("-")

			# Get the indices for target and source and add 1 to them (to get the conllu indices)
			current_t_index = int(current_pair[1]) + 1
			current_s_index = int(current_pair[0]) + 1

			# Check whether the target index is already aligned to anything (a case of 1-to-many alignment),
			# if not, save it as a key and save the source index as value.
			if current_sentence_align.get(current_t_index, None) == None:
				current_sentence_align[current_t_index] = str(current_s_index)
			# If the index was already aligned to a previous source word, add the additional source word alignment as a string
			# (result: {0: "1, 2"))
			else:
				current_sentence_align[current_t_index] += str(", ")
				current_sentence_align[current_t_index] += str(current_s_index)

		aligns_list_target_dict_list.append(current_sentence_align)

	return aligns_list_target_dict_list

correct_proper_nouns()

In [38]:
def correct_proper_nouns(lang_code, main_path):
	"""
	This function takes the translated text and the source text, aligns words with eflomal and corrects proper nouns.
	It takes the dataframe that was created in the function extract_text() and to which the translation was added
	in the function translate().

	To use eflomal, you need to install it first:
	!git clone https://github.com/robertostling/eflomal
	%cd eflomal
	!make
	!sudo make install
	!python3 setup.py install

	In case you don't have sudo permission, you can skip !sudo make install. I did, and I also used a virtual environment (venv), and managed to install eflomal.

	Args:
	- lang_code: the lang code that is used in the names of the files, it should be the same as for extract_text()
	"""
	import pandas as pd
	import re
	import ast
	from IPython.display import display

	# Open the file, created in the previous step
	translated_tokenized_dataframe_path = "{}/results/{}/ParlaMint-{}-translated-tokenized.csv".format(main_path,lang_code, lang_code)

	df = pd.read_csv("{}".format(translated_tokenized_dataframe_path), sep="\t", index_col=0)

	# Move into the eflomal folder
	%cd /home/tajak/Parlamint-translation/eflomal

	# Then we need to create files for all texts and all translations
	source_sentences = open("source_sentences.txt", "w")
	English_sentences = open("English_sentences.txt", "w")

	for i in df["tokenized_text"].to_list():
		source_sentences.write(i)
		source_sentences.write("\n")

	for i in df["translation-tokenized"].to_list():
		English_sentences.write(i)
		English_sentences.write("\n")

	source_sentences.close()
	English_sentences.close()

	# Align sentences with eflomal and get out a file with alignments
	!python3 align.py -s source_sentences.txt -t English_sentences.txt --model 3 -r source-en.rev -f source-en.fwd

	# Create a list of dictionaries of alignments from the returned files which will be added to the final conllu for each word
	forward_alignment_dict_list = alignment_file_to_target_dict("source-en.fwd")
	backward_alignment_dict_list = alignment_file_to_target_dict("source-en.rev")

	# Add to the df
	df["fwd_align_dict"] = forward_alignment_dict_list
	df["bwd_align_dict"] = backward_alignment_dict_list

	# Create forward target alignments from the source alignment direction (by changing the direction in the rev file)
	aligns_list = open("source-en.rev", "r").readlines()
	aligns_list = [i.replace("\n", "") for i in aligns_list]

	# Continue with processing the list to create the final alignments format which I'll use to correct proper names
	aligns_list = [i.split(" ") for i in aligns_list]

	for i in aligns_list:
		for pair in i:
			current_pair = pair.split("-")
			i[i.index(pair)] = {int(current_pair[0]): int(current_pair[1])}
	
	final_aligns = []

	# Create a dictionary out of the rev alignments
	for i in aligns_list:
		current_line = {}

		try:
			for element in i:
				a = list(element.items())[0][0]
				b = list(element.items())[0][1]
				current_line[a] = b
		
			# Check whether the number of pairs in the list is the same as number of items
			if len(i) != len(list(current_line.items())):
				print("Not okay:")
				print(i)
				print(current_line)

			final_aligns.append(current_line)
		
		except:
			print("error")
			print(aligns_list.index(i))
			print(i)
			final_aligns.append("Error")
		
	print("Number of aligned sentences: {}".format(len(final_aligns)))

	# Add a to the df
	df["alignments"] = final_aligns

	# Remove the rev and fwd file
	%rm source-en.rev
	%rm source-en.fwd

	# When we open the dataframe file, the dictionaries with proper names changed into strings - Change strings in the column proper_nouns into dictionaries

	df["proper_nouns"] = df.proper_nouns.astype("str")
	df["proper_nouns"] = df.proper_nouns.apply(lambda x: ast.literal_eval(x))

	# Change nan values in the proper_nouns columns
	df = df.fillna(0)

	# Substitute words in the translation based on alignments
	intermediate_list = list(zip(df["translation-tokenized"], df["proper_nouns"], df["alignments"]))

	new_translations = []
	substituted_all_info = []
	substituted_only = []
	substituted_words = []

	# Add information whether an error occurred
	error_list = []

	for i in intermediate_list:
		current_substituted_list = []
		current_substituted_only = []
		current_substituted_words = {}
		current_error = "No"

		# If no proper names were detected, do not change the translation
		if i[1] == 0:
			new_translations.append(i[0])
		
		else:
			current_translation = i[0]

			# Substitute the word with the source lemma based on the index - loop through the proper nouns to be changed
			for word_index in list(i[1].keys()):
				try:
					# split the translation into list of words
					word_list = current_translation.split()

					# Get index of the substituted word
					substituted_word_index = i[2][word_index]

					# Get the lemma to substitute the word with
					correct_lemma = i[1][word_index][1]

					# If the substitute word and lemma are not the same, get substituted word and its match
					if word_list[substituted_word_index] != correct_lemma:
						current_substituted_list.append((word_list[substituted_word_index], correct_lemma))
						current_substituted_only.append((word_list[substituted_word_index], correct_lemma))

						# Save information on which word was substituted with its conllu index (index + 1) as the key
						current_substituted_words[int(substituted_word_index+1)] = word_list[substituted_word_index]

						# Substitute the word in the word list
						word_list[substituted_word_index] = correct_lemma
					
					else:
						# Add information that substitution was not performed
						current_substituted_list.append(f"No substitution: {word_list[substituted_word_index], correct_lemma}")
					
					# Change the translation by merging the words back into a string
					current_translation = " ".join(word_list)

				except:
					print(f"Issue: index {word_index}: {i[1][word_index]}")
					current_error = f"Issue: index {word_index}: {i[1][word_index]}"

			# After the loop through proper nouns, save the new translation
			new_translations.append(current_translation)
		
		# Add information on what was substituted
		if len(substituted_all_info) != 0:
			substituted_all_info.append(current_substituted_list)
		else:
			substituted_all_info.append(0)

		if len(current_substituted_only) != 0:
			substituted_only.append(current_substituted_only)
		else:
			substituted_only.append(0)

		error_list.append(current_error)

		substituted_words.append(current_substituted_words)


	# Add to the df
	df["new_translations"] = new_translations
	df["substitution_info"] = substituted_all_info
	df["substituted_pairs"] = substituted_only
	df["substituted_words"] = substituted_words
	df["errors"] = error_list

	# Change the working directory once again
	%cd ..

	# Add the word list with indices to the df
	tokenized_text_list = df.tokenized_text.to_list()
	tokenized_text_list = [i.split(" ") for i in tokenized_text_list]
	tokenized_text_dict_list = []

	for sentence in tokenized_text_list:
		sentence_list = []
		counter = 1
		for word in sentence:
			sentence_list.append([word, counter])
			counter += 1
		tokenized_text_dict_list.append(sentence_list)

	df["source_indices"] = tokenized_text_dict_list

	# Save the df
	final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)
	df.to_csv("{}".format(final_dataframe), sep="\t")

	# Display most common substitutions
	df_substituted = df[df["proper_nouns"] != "0"]
	display(df_substituted.substituted_pairs.value_counts()[:20])

	return df

create_conllu()

In [1]:
def create_conllu(file, lang_code, main_path, nlp):
	"""
	The function takes the dataframe (df), created in previous steps and takes only the instances from the df that belong
	to the file that is in the argument. It linguistically processes the translated sentences from the file and saves the file.
	Then we add additional information (metadata and NER annotations) to it with the conllu parser and save the final conllu file.

	Args:
		- file (str): file name from the files list (see above)
		- lang_code (str): the lang code that is used in the names of the files, it should be the same as for extract_text()
	"""

	# Process all sentences in the dataframe and save them to a conllu file
	from stanza.utils.conll import CoNLL
	import stanza
	from conllu import parse
	import ast
	import regex as re
	import os
	import pandas as pd

	final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)

	# Use the dataframe, created in previous steps
	df = pd.read_csv("{}".format(final_dataframe), sep="\t", index_col = 0)

	# Filter out only instances from the file in question
	df = df[df["file"] == file]

	# Add information on the target path
	df["target_path"] = df.file_path.str.replace("Source-data", "Final-data")

	# Get target path
	target_path = list(df.target_path.unique())[0]

	# When we open the dataframe file, the lists and dictionaries turn into strings - change them back
	for column in ["space-after-information", 'fwd_align_dict', 'bwd_align_dict', 'substituted_words', "source_indices"]:
		df[column] = df[column].astype("str")
		df[column] = df[column].apply(lambda x: ast.literal_eval(x))

	# Create lists of information that we need to add to the conllu file
	ids_list = df.sentence_id.to_list()
	source_text = df.text.to_list()
	initial_translation = df.translation.to_list()
	space_after_list = df["space-after-information"].to_list()
	fwd_align_list = df['fwd_align_dict'].to_list()
	bwd_align_list = df['bwd_align_dict'].to_list()
	substituted_words_list = df['substituted_words'].to_list()
	tokenized_text_list = df["source_indices"].to_list()
	sentence_list = df.new_translations.to_list()

	# To feed the entire list into the pipeline, we need to create lists of tokens, split by space
	sentence_list = [x.split(" ") for x in sentence_list]
	
	# Linguistically process the list
	doc = nlp(sentence_list)

	# Save the conllu file
	CoNLL.write_doc2conll(doc, "{}/results/{}/temp/{}".format(main_path, lang_code, file))

	print("{} processed and saved.".format(file))

	# Open the CONLL-u file with the CONLL-u parser

	data = open("{}/results/{}/temp/{}".format(main_path, lang_code, file), "r").read()

	sentences = parse(data)

	# Adding additional information to the conllu
	for sentence in sentences:
		# Get the sentence index
		sentence_index = sentences.index(sentence)

		# Add metadata
		sentence.metadata["sent_id"] = ids_list[sentence_index]
		sentence.metadata["source"] = source_text[sentence_index]
		sentence.metadata["source_indices"] = tokenized_text_list[sentence_index]
		sentence.metadata["initial_translation"] = initial_translation[sentence_index]

		# Delete the current metadata for text
		del sentence.metadata["text"]

		new_translation_text = ""

		# Iterate through tokens
		for word in sentence:
			word_index = sentence.index(word)
			word_conllu_index = word["id"]

			# Check whether the word conllu index (word id) is in the substituted_words_list (it is if it was substituted)
			# If it is, add information on the original translated word
			if substituted_words_list[sentence_index].get(word_conllu_index, None) != None:
				word["misc"]["Translated"] = substituted_words_list[sentence_index][word_conllu_index]
			
			# Do the same for the forward and backward alignment
			if fwd_align_list[sentence_index].get(word_conllu_index, None) != None:
				word["misc"]["ForwardAlignment"] = fwd_align_list[sentence_index][word_conllu_index]

			if bwd_align_list[sentence_index].get(word_conllu_index, None) != None:
				word["misc"]["BackwardAlignment"] = bwd_align_list[sentence_index][word_conllu_index]

			# Remove information on start_char and end_char from the annotation
			del word["misc"]["start_char"]
			del word["misc"]["end_char"]
			
			# Change the NER tags so that they are the same as in the source
			current_ner = word["misc"]["ner"]
			del word["misc"]["ner"]
			
			# Substitute parts of the tags so that they are tha same as in source
			current_ner = re.sub("S-", "B-", current_ner)
			current_ner = re.sub("E-", "I-", current_ner)

			word["misc"]["NER"] = current_ner

			# Get information about the space after based on the index
			current_space_after = space_after_list[sentence_index][word_index]

		# Create new text from translation, correcting the spaces around words
		# based on the SpaceAfter information
			if current_space_after == "No":
				word["misc"]["SpaceAfter"] = "No"
				new_translation_text += word["form"]
			elif current_space_after == "Last":
				new_translation_text += word["form"]
			else:
				new_translation_text += word["form"]
				new_translation_text += " "
		
		sentence.metadata["text"] = new_translation_text
	
	# Create a new conllu file with the updated information

	#final_file = open("{}/results/{}/final_translated_conllu/{}".format(main_path,lang_code, file), "w")
	os.makedirs(os.path.dirname(target_path), exist_ok=True)
	final_file = open("{}".format(target_path), "w")

	for sentence in sentences:
		final_file.write(sentence.serialize())
	
	final_file.close()

	print("Final file {} is saved.".format(target_path))

produce_final_conllu()

In [ ]:
def produce_final_conllu(lang_code, main_path):
	import pandas as pd
	import stanza
	
	# Open df
	final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)

	df = pd.read_csv("{}".format(final_dataframe), sep="\t", index_col=0)

	# Create a list of files
	files = list(df.file.unique())

	# Define the pipeline, instruct it to use a specific package: 	CoNLL03
	nlp = stanza.Pipeline(lang='en', processors="tokenize,mwt,pos,lemma,ner", package={"ner": ["conll03"]}, tokenize_pretokenized=True)

	# Test file
	#for file in ["ParlaMint-CZ_2013-11-25-ps2013-001-01-002-002.conllu"]:
	#	create_conllu(file, lang_code)

	for file in files:
		create_conllu(file, lang_code, main_path, nlp)

# Main Code

In [58]:
from conllu import parse
import pandas as pd
from IPython.display import display
import os
import zipfile
import utils

In [ ]:
# Unzip the folder with the files
#with zipfile.ZipFile("/home/tajak/Parlamint-translation/ParlaMint-CZ/ParlaMint-CZ.conllu.zip", 'r') as zip_ref:
#    zip_ref.extractall("/home/tajak/Parlamint-translation/ParlaMint-CZ/ParlaMint-CZ.conllu")

In [59]:
# Define the language code, used in the file names
lang_code = "CZ"

# Main path
main_path = "/home/tajak/Parlamint-translation"

# Define the translation model to be used
opus_lang_code = "cs"

# Check whether the path to the folder with conllu files is ok
path = "{}/Source-data/ParlaMint-{}.conllu/ParlaMint-{}.conllu".format(main_path, lang_code, lang_code)

# Create a folder with results for this language, e.g. results/CZ
%mkdir results/CZ

# Create (manually) a "temp" folder inside the results/CZ

# Define other paths
extracted_dataframe_path = "{}/results/{}/ParlaMint-{}-extracted-source-data.csv".format(main_path, lang_code, lang_code)

translated_dataframe_path = "{}/results/{}/ParlaMint-{}-translated.csv".format(main_path, lang_code, lang_code)
translated_tokenized_dataframe_path = "{}/results/{}/ParlaMint-{}-translated-tokenized.csv".format(main_path,lang_code, lang_code)
final_dataframe = "{}/results/{}/ParlaMint-{}-final-dataframe.csv".format(main_path,lang_code, lang_code)

mkdir: cannot create directory ‘results/CZ’: File exists


In [60]:
# Extract a list with paths to conllu files and a list with their names
parl_list = []
file_name_list = []

for dir1 in os.listdir(path):
    full_path = os.path.join(path, dir1)
    if os.path.isdir(full_path):
        current = os.listdir(full_path)
        # Keep only files with parliamentary sessions:
        for file in current:
            if "ParlaMint-{}_".format(lang_code) in file:
                if ".conllu" in file:
                    final_path = "{}/{}".format(full_path, file)
                    parl_list.append(final_path)
                    file_name_list.append(file)

print(parl_list[:2])
print(file_name_list[:2])

# See how many files we have:
len(parl_list)

['/home/tajak/Parlamint-translation/Source-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2013/ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021.conllu', '/home/tajak/Parlamint-translation/Source-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2013/ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022.conllu']
['ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021.conllu', 'ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022.conllu']


6328

### Extract information from CONLL-U files

conllu_to_df()

In [61]:
df = conllu_to_df(parl_list[:5], file_name_list[:5], main_path, lang_code)

df.head()

Number of words in the corpora: 13762
File is separated into 3 batches, sizes of batches (in no. of files): 2, 2, 1.
Dataframe saved as /home/tajak/Parlamint-translation/results/CZ/ParlaMint-CZ-extracted-source-data.csv
|        | file_path                                                                                                                                       | file                                                 | sentence_id                                            | text    | tokenized_text   | proper_nouns   |   length |   batch |
|:-------|:------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------|:-------------------------------------------------------|:--------|:-----------------|:---------------|---------:|--------:|
| count  | 956                                                                                              

,file_path,file,sentence_id,text,tokenized_text,proper_nouns,length,batch
0,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,21.,21 .,{},1,1
1,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,Návrh termínu 2. schůze Poslanecké sněmovny,Návrh termínu 2 . schůze Poslanecké sněmovny,{},6,1
2,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,"Vážené paní poslankyně a páni poslanci, připra...","Vážené paní poslankyně a páni poslanci , připr...",{},19,1
3,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,Protože tento výbor bude ustaven až na 2. schů...,Protože tento výbor bude ustaven až na 2 . sch...,{},26,1
4,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,"Chtěl bych upozornit, že pracovní návrh 2. sch...","Chtěl bych upozornit , že pracovní návrh 2 . s...",{},15,1


### Translate

We need to translate the following corpora into English:
- Belgian (BE) - which language??
- Bulgarian (BG)
- Croatian (HR) - We will use "South Slavic MT" based on the manual analysis
- Czech (CZ)
- Danish (DK)
- Dutch (NL)
- French (FR)
- Hungarian (HU) - multilingual model only
- Icelandic (IS)
- Italian (IT)
- Latvian (LV)
- Lithuanian (LT)
- Polish (PL)
- Slovenian (SI) - We will use "Slavic MT" based on the results of the manual analysis
- Spanish? (ES)
- Turkish (TR)
- Austrian (AT)
- Basque (ES-PV)
- Bosnian (BA)
- Catalan (ES-CT)
- Estonian (EE)
- Finnish (FI)
- Galician (ES-GA)
- Greek (GR)
- Norwegian (NO) - NO OPUS-MT model (!) - we can use GT or eTranslation
- Portuguese (PT)
- Romanian (RO)
- Serbian (RS)
- Swedish (SE)
- Ukrainian (UA)

choose_model()

In [22]:
df = choose_model(lang_code, main_path)

Entire corpus has 956 sentences and 13762 words.
Sample files has 20 sentences and 246 words.


/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/tajak/Parlamint-translation/parl

The file is saved as/home/tajak/Parlamint-translation/ results/CZ/ParlaMint-CZ-sample-model-comparison.csv. 


In [23]:
# Then open the sample and manually evaluate which model is better in the column "comparison"
# Open the analysed sample

sample = pd.read_csv("/home/tajak/Parlamint-translation/results/{}/ParlaMint-{}-sample-model-comparison.csv".format(lang_code, lang_code), index_col = 0)
sample.head(2)

,file_path,text,translation-cs,translation-sla,translation-zlw
0,/home/tajak/Parlamint-translation/Source-data/...,21.,21.,21.,21.
1,/home/tajak/Parlamint-translation/Source-data/...,Návrh termínu 2. schůze Poslanecké sněmovny,Draft date of the 2nd meeting of the Chamber o...,Draft deadline 2nd meeting of the Chamber of A...,Proposal for a 2nd meeting of the Chamber of D...


In [ ]:
sample.comparison.value_counts()

The best model for Czech was shown to be cs.

translate()

In [24]:
df = translate(lang_code, opus_lang_code, main_path)

Entire corpus has 956 sentences and 13762 words.


/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


,file_path,file,sentence_id,text,tokenized_text,proper_nouns,length,translation
0,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,21.,21 .,{},1,21.
1,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,Návrh termínu 2. schůze Poslanecké sněmovny,Návrh termínu 2 . schůze Poslanecké sněmovny,{},6,Draft date of the 2nd meeting of the Chamber o...
2,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,"Vážené paní poslankyně a páni poslanci, připra...","Vážené paní poslankyně a páni poslanci , připr...",{},19,"The honourable Members, prepare the House meet..."


The file is saved as /home/tajak/Parlamint-translation/results/CZ/ParlaMint-CZ-translated.csv


## Word alignment

tokenize_translation()

In [27]:
df = tokenize_translation(lang_code, main_path)

df.head(3)

2023-01-25 15:29:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-01-25 15:29:13 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2023-01-25 15:29:13 INFO: Use device: gpu
2023-01-25 15:29:13 INFO: Loading: tokenize
2023-01-25 15:29:13 INFO: Done loading processors!


File saved as /home/tajak/Parlamint-translation/results/CZ/ParlaMint-CZ-translated-tokenized.csv


,file_path,file,sentence_id,text,tokenized_text,proper_nouns,length,translation,translation-tokenized,space-after-information
0,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,21.,21 .,{},1,21.,21 .,"[No, Last]"
1,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,Návrh termínu 2. schůze Poslanecké sněmovny,Návrh termínu 2 . schůze Poslanecké sněmovny,{},6,Draft date of the 2nd meeting of the Chamber o...,Draft date of the 2nd meeting of the Chamber o...,"[Yes, Yes, Yes, Yes, Yes, Yes, Yes, Yes, Yes, ..."
2,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021....,"Vážené paní poslankyně a páni poslanci, připra...","Vážené paní poslankyně a páni poslanci , připr...",{},19,"The honourable Members, prepare the House meet...","The honourable Members , prepare the House mee...","[Yes, Yes, No, Yes, Yes, Yes, Yes, Yes, Yes, Y..."


In [43]:
df = correct_proper_nouns(lang_code, main_path)

# See if there were any errors in word substitution
print(df[df["errors"]!="No"].shape)

# See example of sentences with substituted words
df[df["substituted_pairs"]!= 0][:2]

/home/tajak/Parlamint-translation/eflomal
Number of aligned sentences: 956
/home/tajak/Parlamint-translation


0                                       894
[(Faltynek, Faltýnek)]                    3
[(Jerome, Jeroným)]                       3
[(Karl, Karel)]                           3
[(Titanic, Titanik)]                      2
[(Krakora, Krákora)]                      2
[(Klašek, Klaška)]                        2
[(Foldyn, Foldyna)]                       2
[(Mark, Marková)]                         2
[(Zlatuska, Zlatuška)]                    2
[(Bendlo, Bendl)]                         1
[(Vyzul, Vyzula)]                         1
[(forward, Julínek)]                      1
[(Louis, Ludvík), (Hovork, Hovorka)]      1
[(Zemanovs, Zemanovec)]                   1
[(Petra, Petr), (Kořek, Kořenko)]         1
[(Nykl, Igor), (Igor, Nykl)]              1
[(Kořek, Kořenko)]                        1
[(Kořek, Kořenek)]                        1
[(Nekla, Nekl)]                           1
Name: substituted_pairs, dtype: int64

(0, 19)


,file_path,file,sentence_id,text,tokenized_text,proper_nouns,length,translation,translation-tokenized,space-after-information,fwd_align_dict,bwd_align_dict,alignments,new_translations,substitution_info,substituted_pairs,substituted_words,errors,source_indices
60,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022....,ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022....,Nyní bych požádal pana poslance Jeronýma Tejce...,Nyní bych požádal pana poslance Jeronýma Tejce...,"{5: ['Jeronýma', 'Jeroným'], 6: ['Tejce', 'Tej...",18,I would now ask Mr Jerome Tejc as President of...,I would now ask Mr Jerome Tejc as President of...,"['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Ye...","{2: '2', 3: '1', 4: '4', 5: '5', 6: '6', 7: '7...","{3: '1', 2: '2', 4: '3, 4', 5: '5', 6: '6', 7:...","{0: 2, 1: 1, 2: 3, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...",I would now ask Mr Jeroným Tejc as President o...,"[(Jerome, Jeroným), No substitution: ('Tejc', ...","[(Jerome, Jeroným)]",{6: 'Jerome'},No,"[[Nyní, 1], [bych, 2], [požádal, 3], [pana, 4]..."
62,/home/tajak/Parlamint-translation/Source-data/...,ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022....,ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022....,"Vážený pane předsedající, vážené kolegyně a ko...","Vážený pane předsedající , vážené kolegyně a k...","{14: ['Stanislava', 'Stanislav'], 15: ['Grospi...",14,"Mr President, ladies and gentlemen, I would li...","Mr President , ladies and gentlemen , I would ...","['Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes'...","{1: '2', 2: '3', 3: '4', 4: '5', 5: '7', 6: '8...","{1: '1', 2: '3', 3: '4', 4: '5, 6', 5: '7', 6:...","{0: 0, 2: 1, 3: 2, 4: 3, 5: 3, 6: 4, 7: 5, 8: ...","Mr President , ladies and gentlemen , I would ...","[No substitution: ('Stanislav', 'Stanislav'), ...","[(Grospic, Grospič)]",{15: 'Grospic'},No,"[[Vážený, 1], [pane, 2], [předsedající, 3], [,..."


## Linguistic processing of translated text

In [6]:
produce_final_conllu(lang_code, main_path)

/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-25 16:18:35 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-01-25 16:18:36 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-01-25 16:18:36 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| ner       | conll03  |

2023-01-25 16:18:36 INFO: Use device: gpu
2023-01-25 16:18:36 INFO: Loading: tokenize
2023-01-25 16:18:36 INFO: Loading: pos
2023-01-25 16:18:39 INFO: Loading: lemma
2023-01-25 16:18:39

ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021.conllu processed and saved.
Final file /home/tajak/Parlamint-translation/Final-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2013/ParlaMint-CZ_2013-11-27-ps2013-001-02-016-021.conllu is saved.
ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022.conllu processed and saved.
Final file /home/tajak/Parlamint-translation/Final-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2013/ParlaMint-CZ_2013-12-10-ps2013-004-01-014-022.conllu is saved.
ParlaMint-CZ_2013-12-06-ps2013-002-02-000-000.conllu processed and saved.
Final file /home/tajak/Parlamint-translation/Final-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2013/ParlaMint-CZ_2013-12-06-ps2013-002-02-000-000.conllu is saved.
ParlaMint-CZ_2013-12-06-ps2013-002-02-001-007.conllu processed and saved.
Final file /home/tajak/Parlamint-translation/Final-data/ParlaMint-CZ.conllu/ParlaMint-CZ.conllu/2013/ParlaMint-CZ_2013-12-06-ps2013-002-02-001-007.conllu is saved.
ParlaMint-CZ_2013-11-27-ps2013-001-02-014-017.co